# Documentation QA using Zilliz Cloud and LlamaIndex

This guide demonstrates how to integrate LlamaIndex with Zilliz Cloud to retrieve information from designated sources.

## Before you start

For this example, we are going to be using pymilvus to connect to use Zilliz Cloud and llama-index to handle the data manipulation and pipelining. This example will also require having an OpenAI API key for embedding generation.

In [ ]:
%pip install pymilvus llama-index llama-cpp-python

## Prepare data

We are going to use git to pull the Milvus website data. A majority of the documents come in the form of markdown files.

In [ ]:
!git clone https://github.com/milvus-io/milvus-docs

## Parameters

Here we can find the main parameters that need to be modified for running with your own accounts. Beside each is a description of what it is.

In [2]:
from os import environ
from glob import glob
from llama_index import download_loader, VectorStoreIndex, ServiceContext
from llama_index.vector_stores import MilvusVectorStore
from llama_index.embeddings import HuggingFaceEmbedding

# Set up arguments

# 1. Set up the name of the collection to be created.
COLLECTION_NAME = 'document_qa_db'

# 2. Set up the dimension of the embeddings.
DIMENSION = 1536

# 3. Set the inference parameters
BATCH_SIZE = 128
TOP_K = 3

# 4. Set up the connection parameters for your Zilliz Cloud cluster.
URI = 'YOUR_CLUSTER_ENDPOINT'

TOKEN = 'YOUR_CLUSTER_TOKEN'

# OpenAI API key
environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"
environ["TOKENIZERS_PARALLELISM"] = "false"


## Consume Knowledge

Once we have our data on the system we can proceed to consume it using LlamaIndex and upload it to Zilliz Cloud. This comes in the form of two steps. We begin by loading in a markdown reader from Llama Hub and converting all our markdowns into documents.

In [3]:
# Load the markdown reader from the hub
MarkdownReader = download_loader("MarkdownReader")
markdownreader = MarkdownReader()

# Grab all markdown files and convert them using the reader
docs = []
for file in glob("../../../repos/milvus-docs/site/en/**/*.md", recursive=True):
    docs.extend(markdownreader.load_data(file=file))
print(len(docs))

1613


Once we have our documents formed, we can proceed to push them through into Zilliz Cloud. This step requires the configs for both Zilliz Cloud and OpenAI.

In [4]:
# Push all markdown files into Zilliz Cloud
vector_store = MilvusVectorStore(
    uri=URI, 
    token=TOKEN, 
    collection_name=COLLECTION_NAME, 
    similarity_metric="L2",
    dim=DIMENSION,
)

embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L12-v2")
service_context = ServiceContext.from_defaults(embed_model=embed_model)

index = VectorStoreIndex.from_documents(
    documents=docs, 
    service_context=service_context,
    show_progress=True
)

/Users/anthony/Documents/Github/zdoc-demos/python/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating embeddings: 100%|██████████| 1716/1716 [05:26<00:00,  5.26it/s]


## Ask question

With our documents loaded into Zilliz Cloud, we can begin asking questions. The questions will be searched against the knowledge base and any relevant documents will be used to generate an answer.

In [5]:
query_engine = index.as_query_engine()
response = query_engine.query("What is IVF_FLAT?")
print(str(response))

IVF_FLAT is an index used in Milvus that divides vector space into list clusters. It compares the distances between the target vector and the centroids of all the clusters to return the nearest clusters. Then, it compares the distances between the target vector and the vectors in the selected clusters to find the nearest vectors. IVF_FLAT has performance advantages over FLAT when the number of vectors exceeds a certain threshold.
